Import Statements

In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import sqlite3 as sql3
import re

In [ ]:
sqlite_file = 'IL_SAMHSA.db'
conn = sql3.connect(sqlite_file)

load in csv file with SAMHSA listings and write to SQLite

In [ ]:

df_samhsa = pd.read_csv('IL_Behavioral_Health_Treament_Facility_listing_2017.csv')
df_samhsa.to_sql('facilities', conn, if_exists='replace', index_label ='FACID')
conn.commit()


set up a cursor - read back just the facility ids and websites

In [ ]:
c = conn.cursor()
c2 = conn.cursor()
c.execute('SELECT FACID, website FROM facilities;')
all_rows = c.fetchall()


create new table for the "about us" text for these facilities

In [ ]:
c.execute ('CREATE TABLE abouttext (FACID integer, abouturl text, textfromurl text );')
conn.commit()



In [ ]:
c.execute ('CREATE TABLE badurl (FACID integer, level text, badurl text );')
conn.commit()

In [ ]:
c.execute ('CREATE TABLE maintext (FACID integer, maintext text, url1 text, url2 text, url3 text, url4 text, url5 text );')
conn.commit()

Read each facility's website url, pull the soup, look for a tags that link to "about me" or mission or vision pages
insert appropriate pages into the new text table in the database

In [ ]:

about = ['about', 'who we are', 'mission', 'vision']
tags = ["p", "h1", "h2", "h3", "h4", "h5", "h6"]

testrow = all_rows[0:3]

for row in all_rows:
    
    thisfacid = row[0]
    url = row[1]
    # print (url)
    if url is not None:
        try:
            page = requests.get(url)
        except requests.exceptions.RequestException as e: 
            print e
            level = "top"
            c2.execute ('SELECT COUNT(FACID) FROM badurl WHERE badurl LIKE :url AND FACID = :id', {"url":url, "id":thisfacid})
            dup = c2.fetchone()
            if dup[0] == 0:
                c.execute ('''INSERT INTO badurl 
                              (FACID, level, badurl) 
                              VALUES ( ?, ?, ?);''', (thisfacid, level, url))
                conn.commit()
        else:    
            soup = BeautifulSoup(page.text, "html.parser")
            atags = soup.find_all("a", href = True)
            url_list = []
            rangelen = 5;
            if len(atags) < 5:
                rangelen = len(atags)
            for i in range(rangelen):
                # print(atags[i].get('href'))
                url_list.append(atags[i].get('href'))
                # print(url_list)
            for i in range (rangelen-1, 4):
                url_list.append(" ")
            # print(url_list)
            maintxtlist = soup.find_all(tags)
            maintext =""
            for item in maintxtlist:
                maintext += (item.text + " ")
            c.execute ('''INSERT INTO maintext 
                        (FACID, maintext, url1, url2, url3, url4, url5) 
                        VALUES ( ?, ?, ?, ?, ?, ?, ?);''', (thisfacid, maintext, url_list[0], url_list[1], url_list[2], url_list[3], url_list[4]))
            conn.commit() 
            for link in atags:
                atext = link.text
                for ab in about:
                        if atext.lower().find(ab.lower()) != -1:
                            goodurl = link.get('href')
                            if goodurl is not None:
                                main = ""
                                spawn = ""
                                re1='.*?' # Non-greedy match on filler
                                re2='((?:[a-z][a-z\\.\\d\\-]+)\\.(?:[a-z][a-z\\-]+))(?![\\w\\.])' # Fully Qualified Domain Name 1

                                rg = re.compile(re1+re2,re.IGNORECASE|re.DOTALL)
                                regurl = rg.search(url)
                                if regurl:
                                    main = regurl.group(1)
                                regspwn = rg.search(goodurl)
                                if regspwn:
                                    spawn = regspwn.group(1)
                                if spawn == main:
                                    c2.execute ('SELECT COUNT(FACID) FROM abouttext WHERE abouturl LIKE :url AND FACID = :id', {"url":goodurl, "id":thisfacid})
                                    dup = c2.fetchone()
                                    if dup[0] == 0:
                                        try:
                                            aboutpage = requests.get(goodurl)
                                        except requests.exceptions.RequestException as e: 
                                            print e
                                            level = "bottom"
                                            c2.execute ('SELECT COUNT(FACID) FROM badurl WHERE badurl LIKE :url AND FACID = :id', {"url":goodurl, "id":thisfacid})
                                            dup = c2.fetchone()
                                            if dup[0] == 0:
                                                c.execute ('''INSERT INTO badurl 
                                                              (FACID, level, badurl) 
                                                              VALUES ( ?, ?, ?);''', (thisfacid, level, goodurl))
                                                conn.commit()
                                        else:     
                                            moresoup = BeautifulSoup(aboutpage.text, "html.parser")
                                            txtlist = moresoup.find_all(tags)
                                            goodtext =""
                                            for item in txtlist:
                                                goodtext += (item.text + " ")
                                            c.execute ('''INSERT INTO abouttext 
                                                          (FACID, abouturl, textfromurl) 
                                                          VALUES ( ?, ?, ?);''', (thisfacid, goodurl, goodtext))
                                            conn.commit()                


set up united way database

education: https://uw-mc.org/our-partners/organizations-we-fund/funded-agencies-education/
income: https://uw-mc.org/our-partners/organizations-we-fund/funded-agencies-income/
health: https://uw-mc.org/our-partners/organizations-we-fund/funded-agencies-health/
safety net: https://uw-mc.org/our-partners/organizations-we-fund/funded-agencies-safety-net/

In [ ]:
uwsqlite_file = 'IL_unitedway.sqlite'
conn = sql3.connect(uwsqlite_file)
c = conn.cursor()
c2 = conn.cursor()

In [ ]:
c.execute ('''CREATE  TABLE "main"."facilities" ("FACID" INTEGER PRIMARY KEY  AUTOINCREMENT  NOT NULL , "facname" TEXT, "factype" TEXT, "facurl" TEXT, "facloc" TEXT)''')
c.execute ('CREATE TABLE abouttext (FACID integer, abouturl text, textfromurl text );')
c.execute ('CREATE TABLE badurl (FACID integer, level text, badurl text );')
c.execute ('CREATE TABLE maintext (FACID integer, maintext text, url1 text, url2 text, url3 text, url4 text, url5 text );')
conn.commit()


In [ ]:
unitedway = [("education", "https://uw-mc.org/our-partners/organizations-we-fund/funded-agencies-education/"), ("income", "https://uw-mc.org/our-partners/organizations-we-fund/funded-agencies-income/" ), ("health", "https://uw-mc.org/our-partners/organizations-we-fund/funded-agencies-health/"), ("safety net", "https://uw-mc.org/our-partners/organizations-we-fund/funded-agencies-safety-net/")]

for uwurl in unitedway:
    print (uwurl)

    uwpage = requests.get(uwurl[1])
    uwsoup = BeautifulSoup(uwpage.text, "html.parser")
    uwdiv = uwsoup.find("div", {"id": "content"})
    ulist = uwdiv.find("ul")
    litems = ulist.find_all("li")
    for each in litems:
        atag = each.find("a")
        thisfactype = uwurl[0]
        thisfacurl = atag.get('href')
        thisfacname = atag.text
        thisfacloc = atag.next_sibling
        print(thisfacname)
        insertstmt = 'INSERT INTO facilities (facname, factype, facurl, facloc) VALUES (:facname, :factype, :facurl, :facloc);'    
        c.execute (insertstmt, {"facname":thisfacname, "factype":thisfactype, "facurl":thisfacurl, "facloc":thisfacloc})
        conn.commit()


Load UW facilities into all_rows then rerun about scraping routine

In [ ]:
c.execute('SELECT FACID, facurl FROM facilities;')
all_rows = c.fetchall()